In [47]:
import pandas as pd
import calendar

### We'll first illustrate use of the &lt;DataFrame&gt;.append instance method using the names1881 and names1981 csv files.

In [48]:
names_1881 = pd.read_csv('names1881.csv', names = ['name', 'gender', 'count'])
names_1881.head()

,name,gender,count
0,Mary,F,6919
1,Anna,F,2698
2,Emma,F,2034
3,Elizabeth,F,1852
4,Margaret,F,1658


In [49]:
names_1981 = pd.read_csv('names1981.csv', names = ['name', 'gender', 'count'])
names_1981.head()

,name,gender,count
0,Jennifer,F,57032
1,Jessica,F,42519
2,Amanda,F,34370
3,Sarah,F,28162
4,Melissa,F,28003


*We'll now modify both names_1881 and names_1981 by adding a column indicating the year, and then append names_1981 to names_1881 w/ ignore_index = True (this creates the default RangeIndex as opposed to using index values from the two dataframes along the concatenation axis), and extract all rows containing the name 'Morgan'*

In [50]:
names_1881['Year']=1881
names_1981['Year']=1981
names_1881.append(names_1981, ignore_index=True).pipe(lambda x:x.loc[x['name']=='Morgan',:])

,name,gender,count,Year
1283,Morgan,M,23,1881
2096,Morgan,F,1769,1981
14390,Morgan,M,766,1981


### We'll now illustrate the use of the pandas .concat function using the pittsburgh2013_weather csv file

In [51]:
import os
os.listdir()
pittsburghWeather = pd.read_csv('pittsburgh2013_weather.csv', parse_dates = [0], index_col = [0])
pittsburghWeather.head()

,Max TemperatureF,Mean TemperatureF,Min TemperatureF,Max Dew PointF,MeanDew PointF,Min DewpointF,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressureIn,...,Max VisibilityMiles,Mean VisibilityMiles,Min VisibilityMiles,Max Wind SpeedMPH,Mean Wind SpeedMPH,Max Gust SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,32,28,21,30,27,16,100,89,77,30.10,...,10,6,2,10,8,NaN,0.00,8,Snow,277
2013-01-02,25,21,17,14,12,10,77,67,55,30.27,...,10,10,10,14,5,NaN,0.00,4,NaN,272
2013-01-03,32,24,16,19,15,9,77,67,56,30.25,...,10,10,10,17,8,26.0,0.00,3,NaN,229
2013-01-04,30,28,27,21,19,17,75,68,59,30.28,...,10,10,6,23,16,32.0,0.00,4,NaN,250
2013-01-05,34,30,25,23,20,16,75,68,61,30.42,...,10,10,10,16,10,23.0,0.21,5,NaN,221


*We'll first resample the 'Max TemperatureF' column <b>quarterly</b> using max for aggregation and resample the 'Mean TemperatureF' column <b>monthly</b> using mean for aggregation*

In [52]:
quarterlyMaxes = pittsburghWeather.resample('QS')['Max TemperatureF'].max().to_frame()
quarterlyMaxes.reset_index(inplace = True)
quarterlyMaxes['Date']=quarterlyMaxes['Date'].dt.month.apply(lambda x:calendar.month_abbr[x])
quarterlyMaxes = quarterlyMaxes.rename(columns = {'Date':'Month'}).set_index('Month')
quarterlyMaxes

,Max TemperatureF
Month,
Jan,68
Apr,89
Jul,91
Oct,84


In [53]:
monthlyMeans = pittsburghWeather.resample('M')['Mean TemperatureF'].mean().to_frame()
monthlyMeans.reset_index(inplace = True)
monthlyMeans['Date'] = monthlyMeans['Date'].dt.month.apply(lambda x:calendar.month_abbr[x])
months = monthlyMeans['Date'].tolist() #for use when concatenating monthlyMeans
#and quarterlyMaxes
monthlyMeans = monthlyMeans.rename(columns = {'Date':'Month'}).set_index('Month').sort_index()
monthlyMeans

,Mean TemperatureF
Month,
Apr,53.100000
Aug,70.000000
Dec,34.935484
Feb,28.714286
Jan,32.354839
Jul,72.870968
Jun,70.133333
Mar,35.000000
May,62.612903


*We now use the concat function to concatenate the quarterlyMaxes and monthlyMeans dataframes horizontally (by specifying axis =1). Concatenating
horizontally ensures that index values are not repeated.*

*Notice that we used the list 'months' (assigned in the cell above) to reindex
the concatenated dataframe chronologically*

*Notice also that there are missing at indices where simultaneously quarterly maxes lacks data and monthly means has data*

In [54]:
pd.concat([quarterlyMaxes, monthlyMeans],axis = 1).reindex(months)

,Max TemperatureF,Mean TemperatureF
Jan,68.0,32.354839
Feb,NaN,28.714286
Mar,NaN,35.000000
Apr,89.0,53.100000
May,NaN,62.612903
Jun,NaN,70.133333
Jul,91.0,72.870968
Aug,NaN,70.000000
Sep,NaN,63.766667
Oct,84.0,55.451613


### We'll now concatenate 3 data frames horizontally. These data frames are associated bronze_top5.csv, silver_top5.csv, and gold_top5.csv

In [55]:
medal_types = 'bronze silver gold'.split()
medals = []
for m in medal_types:
    fn = '%s_top5.csv'%m
    medals.append(pd.read_csv(fn, skiprows = 1, header = None, names = ['Country', m], index_col = 0))

combined = pd.concat(medals, axis = 1)
combined

,bronze,silver,gold
France,475.0,461.0,NaN
Germany,454.0,NaN,407.0
Italy,NaN,394.0,460.0
Soviet Union,584.0,627.0,838.0
United Kingdom,505.0,591.0,498.0
United States,1052.0,1195.0,2088.0


*Note the NaN values in the dataframe above. These exist because the
index of the resulting dataframe is the union of the indices of the list of dataframes on which concat was invoked.*

*In this particular context, a NaN value means that that country didn't win
any medals of that type- so the value ought to be zero. We'll correct this in the below. We'll also change all the floating point series to int series.<br> <b>It's imperative that the NaN values are removed BEFORE converted the float series to int series</b>*

**Note that the applymap function applies the specified function to every series in the dataframe**

In [56]:
combined.fillna(0).applymap(int)

,bronze,silver,gold
France,475,461,0
Germany,454,0,407
Italy,0,394,460
Soviet Union,584,627,838
United Kingdom,505,591,498
United States,1052,1195,2088


###  Vertical concatenation of dataframes with shared columns

*We'll now concatenate vertically the dataframes that hold data for that csv files bronze_top5.csv, silver_top5.csv, and gold_top5.csv. Note that these 
three data frames have identical columns, but they represent entirely different
data- one represents bronze medals won, another represents silver medals won,
and the last represents gold medals won.*

*So on vertical concatenation, it's useful to create a multi-index- the inner level of which is each original data frame's index and the outer level of which
is a descriptor the dataset (in this case the medal type). This is done below by using 'keys = medal_types'*

*If we concatenated without doing this (i.e. without specifying keys = 'medal_types') we'd get duplicated indices, which is very misleading, since two index values of zero in this case have very different meanings*

In [57]:
medal_types = 'bronze silver gold'.split()
medals = []
for m in medal_types:
    fn = '%s_top5.csv'%m
    medals.append(pd.read_csv(fn))

combined = pd.concat(medals, axis = 0, keys = medal_types)
combined

Country   Total
bronze 0   United States  1052.0
       1    Soviet Union   584.0
       2  United Kingdom   505.0
       3          France   475.0
       4         Germany   454.0
silver 0   United States  1195.0
       1    Soviet Union   627.0
       2  United Kingdom   591.0
       3          France   461.0
       4           Italy   394.0
gold   0   United States  2088.0
       1    Soviet Union   838.0
       2  United Kingdom   498.0
       3           Italy   460.0
       4         Germany   407.0

*Let's now find all medals won by the United Kingdom using slicing of the multi-index*

*This will entail making a few changes to the structure of the dataframe*

**Note: a data frame's multi-index MUST be sorted prior to calling loc on it (here 'it' refers to the data frame**

In [58]:
combined = combined.reset_index().drop('level_1', axis = 1).rename(columns = {'level_0':'medal_type'}).set_index(['medal_type','Country']).sort_index(level = 0).loc[(slice(None), 'United Kingdom'),:]
combined['Total']=combined['Total'].map(int)
combined

,,Total
medal_type,Country,
bronze,United Kingdom,505
gold,United Kingdom,498
silver,United Kingdom,591


### Comparing Horizontal and Vertical Concatenation

**Concatenating the medals horizontally using medal types as keys**

In [59]:
pd.concat(medals, axis = 1, keys = medal_types)

bronze                  silver                    gold        
          Country   Total         Country   Total         Country   Total
0   United States  1052.0   United States  1195.0   United States  2088.0
1    Soviet Union   584.0    Soviet Union   627.0    Soviet Union   838.0
2  United Kingdom   505.0  United Kingdom   591.0  United Kingdom   498.0
3          France   475.0          France   461.0           Italy   460.0
4         Germany   454.0           Italy   394.0         Germany   407.0

*Notice how 'Country' and 'Total' are duplicated. This isn't what we really want. What we really want is for gold, silver
and bronze to be separate columns and the keys to be the country names. We'll really need to use the same strategy that was used a few cells above; this includes having to preprocess each data frame to make the 'Total' column either a 'Gold', 'Silver', or 'Bronze' column.*

In [60]:
medal_types = ['bronze', 'silver','gold']
frames = []
for m in medal_types:
    frames.append(pd.read_csv('%s_top5.csv'%m, skiprows = 1, names =  ['Country', m], index_col = [0] ))
pd.concat(frames, axis = 1).fillna(0).applymap(int)


,bronze,silver,gold
France,475,461,0
Germany,454,0,407
Italy,0,394,460
Soviet Union,584,627,838
United Kingdom,505,591,498
United States,1052,1195,2088


**<i>Now we'll read in 3 files that contain transaction data for hardware, software and service products.  Data for each type of product is in a separate file; they'll be read into 3 separate data frames</i>**

In [61]:
cats = 'Hardware Software Service'.split()
frames = []
for c in cats:
    frames.append(pd.read_csv('feb-sales-%s.csv'%c, index_col = [0], parse_dates= [0]))


In [62]:
frames[0]

,Company,Product,Units
Date,,,
2015-02-04 21:52:45,Acme Coporation,Hardware,14
2015-02-07 22:58:10,Acme Coporation,Hardware,1
2015-02-19 10:59:33,Mediacore,Hardware,16
2015-02-02 20:54:49,Mediacore,Hardware,9
2015-02-21 20:41:47,Hooli,Hardware,3


In [63]:
frames[1]

,Company,Product,Units
Date,,,
2015-02-16 12:09:19,Hooli,Software,10
2015-02-03 14:14:18,Initech,Software,13
2015-02-02 08:33:01,Hooli,Software,3
2015-02-05 01:53:06,Acme Coporation,Software,19
2015-02-11 20:03:08,Initech,Software,7
2015-02-09 13:09:55,Mediacore,Software,7
2015-02-11 22:50:44,Hooli,Software,4
2015-02-04 15:36:29,Streeplex,Software,13
2015-02-21 05:01:26,Mediacore,Software,3


In [64]:
frames[2]

,Company,Product,Units
Date,,,
2015-02-26 08:57:45,Streeplex,Service,4
2015-02-25 00:29:00,Initech,Service,10
2015-02-09 08:57:30,Streeplex,Service,19
2015-02-26 08:58:51,Streeplex,Service,1
2015-02-05 22:05:03,Hooli,Service,10
2015-02-19 16:02:58,Mediacore,Service,10


<i>We'll now concatenate the 3 data frames vertically. The resulting data frame will hold transaction data grouped vertically according to which product category the transaction falls in. <b>This is the intuitive grouping</b><i> 

In [65]:
pd.concat(frames, keys = ['Hardware', 'Software', 'Service'], axis = 0)

Company   Product  Units
         Date                                                 
Hardware 2015-02-04 21:52:45  Acme Coporation  Hardware     14
         2015-02-07 22:58:10  Acme Coporation  Hardware      1
         2015-02-19 10:59:33        Mediacore  Hardware     16
         2015-02-02 20:54:49        Mediacore  Hardware      9
         2015-02-21 20:41:47            Hooli  Hardware      3
Software 2015-02-16 12:09:19            Hooli  Software     10
         2015-02-03 14:14:18          Initech  Software     13
         2015-02-02 08:33:01            Hooli  Software      3
         2015-02-05 01:53:06  Acme Coporation  Software     19
         2015-02-11 20:03:08          Initech  Software      7
         2015-02-09 13:09:55        Mediacore  Software      7
         2015-02-11 22:50:44            Hooli  Software      4
         2015-02-04 15:36:29        Streeplex  Software     13
         2015-02-21 05:01:26        Mediacore  Software      3
Service  2015-02-26 08:57:45        Streeplex   Service      4
         2015-02-25 00:29:00          Initech   Service     10
         2015-02-09 08:57:30        Streeplex   Service     19
         2015-02-26 08:58:51        Streeplex   Service      1
         2015-02-05 22:05:03            Hooli   Service     10
         2015-02-19 16:02:58        Mediacore   Service     10

*We'll now concatenate the 3 data frames horizontally. The resulting data frame will hold transaction data grouped horizontally according to which product category the transaction falls in. <b>Note that this is clearly the worse of the groupings.</b>*

In [66]:
pd.concat(frames, keys = ['Hardware', 'Software', 'Service'], axis = 1)

Hardware                         Software  \
                             Company   Product Units          Company   
Date                                                                    
2015-02-02 08:33:01              NaN       NaN   NaN            Hooli   
2015-02-02 20:54:49        Mediacore  Hardware   9.0              NaN   
2015-02-03 14:14:18              NaN       NaN   NaN          Initech   
2015-02-04 15:36:29              NaN       NaN   NaN        Streeplex   
2015-02-04 21:52:45  Acme Coporation  Hardware  14.0              NaN   
2015-02-05 01:53:06              NaN       NaN   NaN  Acme Coporation   
2015-02-05 22:05:03              NaN       NaN   NaN              NaN   
2015-02-07 22:58:10  Acme Coporation  Hardware   1.0              NaN   
2015-02-09 08:57:30              NaN       NaN   NaN              NaN   
2015-02-09 13:09:55              NaN       NaN   NaN        Mediacore   
2015-02-11 20:03:08              NaN       NaN   NaN          Initech   
2015-02-11 22:50:44              NaN       NaN   NaN            Hooli   
2015-02-16 12:09:19              NaN       NaN   NaN            Hooli   
2015-02-19 10:59:33        Mediacore  Hardware  16.0              NaN   
2015-02-19 16:02:58              NaN       NaN   NaN              NaN   
2015-02-21 05:01:26              NaN       NaN   NaN        Mediacore   
2015-02-21 20:41:47            Hooli  Hardware   3.0              NaN   
2015-02-25 00:29:00              NaN       NaN   NaN              NaN   
2015-02-26 08:57:45              NaN       NaN   NaN              NaN   
2015-02-26 08:58:51              NaN       NaN   NaN              NaN   

                                       Service                 
                      Product Units    Company  Product Units  
Date                                                           
2015-02-02 08:33:01  Software   3.0        NaN      NaN   NaN  
2015-02-02 20:54:49       NaN   NaN        NaN      NaN   NaN  
2015-02-03 14:14:18  Software  13.0        NaN      NaN   NaN  
2015-02-04 15:36:29  Software  13.0        NaN      NaN   NaN  
2015-02-04 21:52:45       NaN   NaN        NaN      NaN   NaN  
2015-02-05 01:53:06  Software  19.0        NaN      NaN   NaN  
2015-02-05 22:05:03       NaN   NaN      Hooli  Service  10.0  
2015-02-07 22:58:10       NaN   NaN        NaN      NaN   NaN  
2015-02-09 08:57:30       NaN   NaN  Streeplex  Service  19.0  
2015-02-09 13:09:55  Software   7.0        NaN      NaN   NaN  
2015-02-11 20:03:08  Software   7.0        NaN      NaN   NaN  
2015-02-11 22:50:44  Software   4.0        NaN      NaN   NaN  
2015-02-16 12:09:19  Software  10.0        NaN      NaN   NaN  
2015-02-19 10:59:33       NaN   NaN        NaN      NaN   NaN  
2015-02-19 16:02:58       NaN   NaN  Mediacore  Service  10.0  
2015-02-21 05:01:26  Software   3.0        NaN      NaN   NaN  
2015-02-21 20:41:47       NaN   NaN        NaN      NaN   NaN  
2015-02-25 00:29:00       NaN   NaN    Initech  Service  10.0  
2015-02-26 08:57:45       NaN   NaN  Streeplex  Service   4.0  
2015-02-26 08:58:51       NaN   NaN  Streeplex  Service   1.0

*It's worse because 'Company', 'Product' and 'Units' are duplicated across the product categories and
this gives our eyes more columns to have to look across. Also since every transaction (i.e. observation) falls into a single product category, you'll have missing values in all of the columns of a transaction that are associated with any category aside from that corresponding to the transaction.*

### Concatenating grouped data frames using a dictionary&lt;str:dataframe&gt; Notice that a multiindex will be created wherein the outer level is the string  key for each key in the dictionary and the inner level is the group in each of the original data frames

We'll be grouping 3 months of sales transaction data by the Company name. There's a data frame for each month; each month's data frame is constructed by reading in a corresponding csv file.

In [67]:
jan_data = pd.read_csv('sales-jan-2015.csv')
feb_data = pd.read_csv('sales-feb-2015.csv')
mar_data = pd.read_csv('sales-mar-2015.csv')
jan_data.head()

,Date,Company,Product,Units
0,2015-01-21 19:13:21,Streeplex,Hardware,11
1,2015-01-09 05:23:51,Streeplex,Service,8
2,2015-01-06 17:19:34,Initech,Hardware,17
3,2015-01-02 09:51:06,Hooli,Hardware,16
4,2015-01-11 14:51:02,Hooli,Hardware,11


All 3 data frames bear the above format.

**Now for each data frame, we'll group it by Company, then aggregate the grouped dataframe using the sum() function. We'll then concatenate the data frame using the dictionary** <em>{'jan':jan_data, 'feb':feb_data, 'mar':mar_data}</em>

In [68]:
jan_data  = jan_data.groupby('Company').sum()
feb_data = feb_data.groupby('Company').sum()
mar_data = mar_data.groupby('Company').sum()
jan_data

,Units
Company,
Acme Coporation,76
Hooli,70
Initech,37
Mediacore,15
Streeplex,50


All the three grouped data frames follow the above format. Notice that now the companies are the keys and the only other column retained was 'Units'.

In [70]:
monthly_unit_sales_by_company = {'jan':jan_data, 'feb':feb_data, 'mar':mar_data}
pd.concat(monthly_unit_sales_by_company)

Units
    Company               
feb Acme Coporation     34
    Hooli               30
    Initech             30
    Mediacore           45
    Streeplex           37
jan Acme Coporation     76
    Hooli               70
    Initech             37
    Mediacore           15
    Streeplex           50
mar Acme Coporation      5
    Hooli               37
    Initech             68
    Mediacore           68
    Streeplex           40

*Notice that the concatenated data frame has a multi-index whose outer levels are the keys in the dictionary passed to pd.concat*